In [4]:
# go into requirements.txt 
# !pip -q install pandas
!pip -q install nltk
# !pip -q install torchtext
# !pip install -q torch 
# !pip install -q pytorch_lightning
# !pip install -q torchvision

In [9]:
import pandas as pd
import random 
import zipfile
# with zipfile.ZipFile('./data/ag-news-classification-dataset.zip') as zf:
#     zf.extractall('./data/')

In [11]:
# datasets.py
trainpath = '../data/train.csv'
testpath = '../data/test.csv'
train = pd.read_csv(trainpath)
test = pd.read_csv(testpath)
train.head(5)

,Class Index,Title,Description
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli..."
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...
4,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco..."


In [17]:
# concatenate Title and Description
# clean and tokenize 
from torchtext.data import get_tokenizer
from nltk.corpus import stopwords
import nltk
import re
try:
    stopwords = set(stopwords.words('english'))
except LookupError as e:
    print(f"{e}, {e.__class__}")
    nltk.download('stopwords')
    stopwords = set(stopwords.words('english'))
    

In [18]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/P76114511/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# utils.py
def clean(x):
    text,label = x['text'], x['label']
    return {'text': re.sub(r'[~`\-!@#$%^&*():;"{}_/?><\|.,`0-9\\]',' ',text), # regex replace multiple spaces to one 
            'label':label}

In [30]:
def preproc(title, descrip):
    text = title + '.' + descrip
    text = re.sub(r'[~`\-!@#$%^&*():;"{}_/?><\|.,`0-9\\]',' ',text)
    text = nltk.word_tokenize(text)     # as suggested by senpai 
    text = [tok for tok in text if tok not in stopwords]
    return text
from torchtext.vocab import build_vocab_from_iterator
def build_vocab(datasets):
      for dataset in datasets:
        for rid, data in dataset.iterrows():
            text = data['preproc_text']
            yield text
tokenizer = get_tokenizer("basic_english")
train['preproc_text'] = train.apply(lambda x:preproc(x['Title'], x['Description']), axis=1)
test['preproc_text'] = test.apply(lambda x:preproc(x['Title'], x['Description']), axis=1)
# class IterableTexts:
#   def __iter__(self):
#     for dataset in [trainset, testset]:
#         for data in dataset:
#             text = data['text']
#             yield list(x.lower() for x in tokenizer(text) if x.lower() not in stopwords)

In [36]:
vocabs = build_vocab_from_iterator(build_vocab([train, test]), 
                                   specials=["[PAD]", "[UNK]"], max_tokens = None)
vocabs.set_default_index(vocabs["[UNK]"]) # let padding be index 0 
print(len(vocabs))
vocabs.get_stoi()
vocabs['[UNK]']

In [38]:
vocabs.get_stoi()
vocabs['[UNK]']

1

In [ ]:
# wrap into dataloaders, collator and shuffling 

In [ ]:
# set up Lightning model architecture 

In [ ]:
# Trainer.fit 